In [31]:
import os
import io
import time
import pickle
import pandas as pd
import numpy as np

import cv2
from skimage import transform
import matplotlib.pyplot as plt
from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torchvision
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP

In [15]:
desired_categories = pd.read_csv('desired_categories.csv')
desired_categories = [row[1] for csv_idx, row in desired_categories.iterrows()]
print(desired_categories)

['Drink', 'Telephone', 'Weapon', 'Tool', 'Container', 'Helmet', 'Racket', 'Ball', 'Sports Equipment_other', 'Musical instrument', 'Medical equipment', 'Personal care', 'Cosmetics', 'Shellfish', 'Fish', 'Reptile&Amphibian', 'Mammal', 'Invertebrate', 'Bird', 'Luggage & bags', 'Footwear', 'Trousers', 'Fashion accessory', 'Glove', 'Hat', 'Clothing_other', 'Aerial vehicle', 'Watercraft', 'Auto part', 'Car', 'Bicycle', 'Land Vehicle_other', 'Flower', 'Tree', 'Vegetable', 'Seafood', 'Baked goods', 'Fruit', 'Desseert&Snack', 'Fast food', 'Food_other', 'Man', 'Woman', 'Boy', 'Girl', 'Human eye', 'Human head', 'Human face', 'Human mouth', 'Human ear', 'Human hair', 'Human hand', 'Human foot', 'Human arm', 'Human leg', 'Human beard', 'Electronics', 'Street signs', 'Sculpture', 'Furniture', 'Kitchen Appliance&Utensil', 'Tableware', 'Office supplies', 'Plumbing fixture', 'Home appliance', 'Toy']


In [16]:
openimages = json.load(open('bbox_labels_600_hierarchy.json'))
csv = pd.read_csv('class-descriptions-boxable.csv')

In [17]:
print(len(openimages["Subcategory"]))
print(csv)

90
       /m/011k07    Tortoise
0    /m/011q46kg   Container
1      /m/012074      Magpie
2      /m/0120dh  Sea turtle
3      /m/01226z    Football
4      /m/012n7d   Ambulance
..           ...         ...
595     /m/0qmmr  Wheelchair
596   /m/0wdt60w  Rugby ball
597      /m/0xfy   Armadillo
598     /m/0xzly     Maracas
599     /m/0zvk5      Helmet

[600 rows x 2 columns]


In [18]:
id2label_dict = {}
label2id_dict = {}
for csv_idx, row in csv.iterrows():
    id2label_dict[row[0]] = row[1]
    label2id_dict[row[1]] = row[0]

In [19]:
output_dict ={}
delete = []

for desired_tag in desired_categories:
    try:
        desired_ID = label2id_dict[desired_tag]
    except KeyError:
        pass
    else:
        output_dict[desired_tag] = [label2id_dict[desired_tag]]
        delete.append(desired_tag)
        
desired_categories = [i for i in desired_categories if i not in delete]
print('Output:', output_dict)
print('Whats left', desired_categories)

Output: {'Drink': ['/m/0271t'], 'Telephone': ['/m/07cx4'], 'Weapon': ['/m/083kb'], 'Tool': ['/m/07k1x'], 'Container': ['/m/011q46kg'], 'Helmet': ['/m/0zvk5'], 'Racket': ['/m/0dv9c'], 'Ball': ['/m/018xm'], 'Musical instrument': ['/m/04szw'], 'Medical equipment': ['/m/01xgg_'], 'Personal care': ['/m/02w3_ws'], 'Cosmetics': ['/m/014trl'], 'Shellfish': ['/m/0fbdv'], 'Fish': ['/m/0ch_cf'], 'Mammal': ['/m/04rky'], 'Invertebrate': ['/m/03xxp'], 'Bird': ['/m/015p6'], 'Footwear': ['/m/09j5n'], 'Trousers': ['/m/07mhn'], 'Fashion accessory': ['/m/0463sg'], 'Glove': ['/m/0174n1'], 'Hat': ['/m/02dl1y'], 'Watercraft': ['/m/01rzcn'], 'Auto part': ['/m/08dz3q'], 'Car': ['/m/0k4j'], 'Bicycle': ['/m/0199g'], 'Flower': ['/m/0c9ph5'], 'Tree': ['/m/07j7r'], 'Vegetable': ['/m/0f4s2w'], 'Seafood': ['/m/06nwz'], 'Baked goods': ['/m/052lwg6'], 'Fruit': ['/m/02xwb'], 'Fast food': ['/m/01_bhs'], 'Man': ['/m/04yx4'], 'Woman': ['/m/03bt1vf'], 'Boy': ['/m/01bl7v'], 'Girl': ['/m/05r655'], 'Human eye': ['/m/014sv8'],

## Homeless tags processing

In [121]:
primary_categories, homeless_categories, delete = np.array([]), np.array([]), []

for i in range(len(openimages["Subcategory"])):
    ID = openimages["Subcategory"][i]['LabelName']
    try:
        sub = openimages["Subcategory"][i]["Subcategory"]
    except KeyError:
        homeless_categories = np.append(homeless_categories, ID)
    else:
        primary_categories = np.append(primary_categories, ID)
        
output_dict ={} # 'Plant':[/m/0zrthkd, /m/0zrpfhj]

for desired_tag in desired_categories:
    try:
        desired_ID = label2id_dict[desired_tag]
    except:
        pass
    else:
        if desired_ID in homeless_categories:
            output_dict[desired_tag] = label2id_dict[desired_tag]
            delete.append(desired_tag)

desired_categories = [i for i in desired_categories if i not in delete]
print('Done processing homeless categories:', output_dict)

Done processing homeless categories: {'Human head': '/m/04hgtk', 'Human mouth': '/m/0283dt1', 'Human ear': '/m/039xj_', 'Human hand': '/m/0k65p', 'Human foot': '/m/031n1', 'Human arm': '/m/0dzf4', 'Human beard': '/m/015h_t'}


In [122]:
for desired_tag in desired_categories:
    try:
        desired_ID = label2id_dict[desired_tag]
    except:
        pass
    else:
        if desired_ID in primary_categories:
            output_dict[desired_tag] = label2id_dict[desired_tag]
            delete.append(desired_tag) 

desired_categories = [i for i in desired_categories if i not in delete]
print('Done processing homeless categories:', output_dict)

Done processing homeless categories: {'Human head': '/m/04hgtk', 'Human mouth': '/m/0283dt1', 'Human ear': '/m/039xj_', 'Human hand': '/m/0k65p', 'Human foot': '/m/031n1', 'Human arm': '/m/0dzf4', 'Human beard': '/m/015h_t', 'Drink': '/m/0271t', 'Telephone': '/m/07cx4', 'Weapon': '/m/083kb', 'Tool': '/m/07k1x', 'Personal care': '/m/02w3_ws', 'Cosmetics': '/m/014trl', 'Sculpture': '/m/06msq', 'Furniture': '/m/0c_jw', 'Office supplies': '/m/02rdsp', 'Plumbing fixture': '/m/02pkr5', 'Home appliance': '/m/019dx1', 'Toy': '/m/0138tl'}


In [123]:
print(desired_categories)

['Container', 'Helmet', 'Racket', 'Ball', 'Sports Equipment_other', 'Musical Instrument', 'Medical Equipment', 'Shellfish', 'Fish', 'Reptile&Amphibian', 'Mammal', 'Invertberate', 'Bird', 'Luggage & bags', 'Footware', 'Trousers', 'Fashion Accessory', 'Glove', 'Hat', 'Clothing_other', 'Aerial veheicle', 'Watercraft', 'Auto Part', 'Car', 'Bicycle', 'Land Vehicle_other', 'Flower', 'Tree', 'Vegetable', 'Seafood', 'Baked goods', 'Fruit', 'Desseert&Snack', 'Fastfood', 'Food_other', 'Man', 'Women', 'Boy', 'Girl', 'Humen eye', 'Human Face', 'Hair', 'Human leg', 'Electronics', 'Street signs', 'Kitchen Appliance&Utensil', 'Tableware']
